<a href="https://colab.research.google.com/github/Suriyanand/GEN_AI_PROJECTS/blob/main/langchain_step_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings , HuggingFaceBgeEmbeddings
from langchain.vectorstores import chroma
import os

In [ ]:
text_loader = TextLoader("/content/example.txt")
text_docs = text_loader.load()
print(text_docs)

[Document(metadata={'source': '/content/example.txt'}, page_content='LangChain is a framework for developing applications powered by language models. It enables the chaining of components like prompt templates, language model calls, and output parsers in a flexible and easy-to-use way.\n\nLangChain supports document loading, splitting, embedding, and storage using vector databases. This makes it suitable for building powerful retrieval-based applications like chatbots, Q&A systems, and search engines.\n\nDevelopers can use embeddings from providers like HuggingFace or OpenAI and persist data into vector stores like Chroma or FAISS for efficient retrieval.\n')]


In [ ]:
pdf_loader = PyPDFLoader("https://arxiv.org/pdf/2305.12675.pdf")
pdf_docs = pdf_loader.load()
print(pdf_docs)

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-02-28T02:26:49+00:00', 'author': '', 'keywords': '', 'moddate': '2024-02-28T02:26:49+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'https://arxiv.org/pdf/2305.12675.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1'}, page_content='A Frustratingly Simple Decoding Method for\nNeural Text Generation\nHaoran Yang♠,∗, Deng Cai♡,†, Huayang Li♣, Wei Bi♡, Wai Lam♠, Shuming Shi♡\n♠The Chinese University of Hong Kong ♡Tencent AI Lab ♣Nara Institute of Science and Technology\n{hryang, wlam}@se.cuhk.edu.hk, li.huayang.lh6@is.naist.jp\n{jcykcai, victoriabi, shumingshi}@tencent.com\nAbstract\nWe introduce a frustratingly simple, highly efficient, and surprisingly effective decoding method, termed Frustratingly\nSimple Decoding (FSD), for neural text generation. Th

In [ ]:
all_docs = text_docs + pdf_docs

In [ ]:
from posixpath import split
spiltter = RecursiveCharacterTextSplitter(chunk_size = 500 ,chunk_overlap = 100)
split_docs = spiltter.split_documents(all_docs)

In [ ]:
hf_embedder = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")
hf_embeddering = hf_embedder.embed_documents([docs.page_content for docs in split_docs])


In [ ]:
print("Hugging_face lenght : " , len(hf_embeddering))

Hugging_face lenght :  229


In [ ]:
from langchain.vectorstores import Chroma
chroma_store = Chroma.from_documents(
    documents=split_docs,
    embedding=hf_embedder,
    persist_directory="chroma_db"
)

In [ ]:
query = "what is Langchain?"
result = chroma_store.similarity_search(query)
for i,res in enumerate(result):
   print(f"\nResult {i+1}:\n{res.page_content[:300]}")


Result 1:
LangChain is a framework for developing applications powered by language models. It enables the chaining of components like prompt templates, language model calls, and output parsers in a flexible and easy-to-use way.

LangChain supports document loading, splitting, embedding, and storage using vect

Result 2:
3. Background
3.1. Language Models
An LM is a probability distribution over token se-
quences. Given a sequence x1:t = x1, x2, . . . , xt
of length t, LM assigns a probability p(x1:t) to the
sequence, which is usually decomposed in an au-
toregressive fashion: p(x1:t) =Qt
i=1 p(xi|x<i).
N-gram Langu

Result 3:
7. References
BigScience. 2023. Bloom: A 176b-parameter open-
access multilingual language model.
Tom B. Brown, Benjamin Mann, Nick Ryder,
Melanie Subbiah, Jared Kaplan, Prafulla Dhari-
wal, Arvind Neelakantan, Pranav Shyam, Girish
Sastry, Amanda Askell, Sandhini Agarwal, Ariel
Herbert-Voss, Gretche

Result 4:
First, all the operations (i.e., construction, predic

In [ ]:
chroma_store.persist()
print("Chroma DB persisted!")

Chroma DB persisted!


<ipython-input-22-fbf7ec0e3050>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma_store.persist()


In [ ]:
loaded_chroma = Chroma(persist_directory="chroma_db", embedding_function=hf_embedder)
print("Chroma DB loaded with", len(loaded_chroma.get()["documents"]), "documents.")

Chroma DB loaded with 229 documents.


<ipython-input-23-952cdc8cb6e4>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  loaded_chroma = Chroma(persist_directory="chroma_db", embedding_function=hf_embedder)
